*Setting up the enviroment*

In [4]:
import subprocess
import os


if not os.path.exists("acnportal"):
    subprocess.run(["git", "clone", "-b", "EHN_stochastic_network", "https://github.com/zach401/acnportal.git"])

if not os.path.exists("adacharge"):
    subprocess.run(["git", "clone", "-b", "py_3.6_compatibility", "https://github.com/caltech-netlab/adacharge.git"])

subprocess.run(["pip", "install",  "-e", "acnportal/."])
subprocess.run(["pip", "install",  "-e", "adacharge/."])

CompletedProcess(args=['pip', 'install', '-e', 'acnportal/.'], returncode=0)

In [2]:
if not os.path.exists("data/jpl_weekeday_40.pkl"):
    if not os.path.exists("data"):
        subprocess.run(["mkdir", "data"])
    subprocess.run(["wget", "-P", "./data", "https://ev.caltech.edu/assets/data/gmm/jpl_weekday_40.pkl"])


# Comparing Infrastructure Designs using ACN-Sim
### by Zachary Lee
#### Last updated: 4/16/2020


In this case study, we demonstrate how ACN-Data and ACN-Sim can be used to evaluate infrastructure configurations and algorithms. We consider the case of a site host who expects to charge approximately 100 EVs per day with a demand pattern similar to that of JPL.

The site host has several options, including  
*   102 Uncontrolled Level-1 EVSEs with a 200 kW Transformer
*   30 Uncontrolled Level-2 EVSEs with a 200 kW Transformer
*   102 Uncontrolled Level-2 EVSEs with a 670 kW Transformer
*   102 Smart Level-2 EVSEs running LLF with a 200 kW Transformer

We evaluate the scenarios on the number of times drivers would have to swap parking places to allow other drivers to charge, the percentage of total demand met, and the operating costs (calculated using ACN-Sim's integration with utility tariffs). This demonstrates the significant benefits of developing smart EV charging systems in terms of reducing both capital costs (transformer capacity) and operating costs.


In [2]:
import acnportal

from copy import deepcopy
import warnings
import pytz
import numpy as np
import pickle
from datetime import datetime
from acnportal import acnsim
from acnportal import algorithms
from acnportal.signals.tariffs.tou_tariff import TimeOfUseTariff
from acnportal.acnsim.events import GaussianMixtureEvents
from acnportal.contrib.acnsim import StochasticNetwork
import adacharge

## Charging Network Designs

To define our charging network options, we will use two functions which generate a StochasticNetwork object. The StochasticNetwork assigns users to spaces dynamically based on available spaces. In this example, we will assume each driver has equal preference for all spots.

If all spaces are taken, drivers join a queue which is drained as drivers finish charging and move their vehicle (the early departure option specifies that drivers move their vehicle when it is done charging rather than their normal departure time). We record each time that the user leave and is replaced with someone from the queue as a swap. Swaps are undesirable as they waste time and are frustrating for users. Despite this, swapping is a common practice in many charging facilities where the number of users exceeds the number of EVSEs.  

In [3]:
def level_1_network(transformer_cap=200, evse_per_phase=34):
    """ Configurable charging network for level-1 EVSEs connected line to ground
        at 120 V. 

    Args:
        transformer_cap (float): Capacity of the transformer feeding the network
          [kW]
        evse_per_phase (int): Number of EVSEs on each phase. Total number of 
          EVSEs will be 3 * evse_per_phase.

    Returns:
        ChargingNetwork: Configured ChargingNetwork.  
    """
    network = StochasticNetwork(early_departure=True)
    voltage = 120

    # Define the sets of EVSEs in the Caltech ACN.
    A_ids = ['A-{0}'.format(i) for i in range(evse_per_phase)]
    B_ids = ['B-{0}'.format(i) for i in range(evse_per_phase)]
    C_ids = ['C-{0}'.format(i) for i in range(evse_per_phase)]

    # Add Caltech EVSEs
    for evse_id in A_ids:
        network.register_evse(acnsim.FiniteRatesEVSE(evse_id, [0, 16]), voltage, 0)
    for evse_id in B_ids:
        network.register_evse(acnsim.FiniteRatesEVSE(evse_id, [0, 16]), voltage, 120)
    for evse_id in C_ids:
        network.register_evse(acnsim.FiniteRatesEVSE(evse_id, [0, 16]), voltage, -120)

    # Add Caltech Constraint Set
    I3a = acnsim.Current(A_ids)
    I3b = acnsim.Current(B_ids)
    I3c = acnsim.Current(C_ids)

    # Define intermediate currents
    I2a = (1 / 4) * (I3a - I3c)
    I2b = (1 / 4) * (I3b - I3a)
    I2c = (1 / 4) * (I3c - I3b)

    # Build constraint set
    primary_side_constr = transformer_cap * 1000 / 3 / 277
    secondary_side_constr = transformer_cap * 1000 / 3 / 120
    network.add_constraint(I3a, secondary_side_constr, name='Secondary A')
    network.add_constraint(I3b, secondary_side_constr, name='Secondary B')
    network.add_constraint(I3c, secondary_side_constr, name='Secondary C')
    network.add_constraint(I2a, primary_side_constr, name='Primary A')
    network.add_constraint(I2b, primary_side_constr, name='Primary B')
    network.add_constraint(I2c, primary_side_constr, name='Primary C')

    return network


def level_2_network(transformer_cap=200, evse_per_phase=34):
    """ Configurable charging network for level-2 EVSEs connected line to line
        at 208 V. 

    Args:
        transformer_cap (float): Capacity of the transformer feeding the network
          [kW]
        evse_per_phase (int): Number of EVSEs on each phase. Total number of 
          EVSEs will be 3 * evse_per_phase.

    Returns:
        ChargingNetwork: Configured ChargingNetwork.  
    """
    network = StochasticNetwork(early_departure=True)
    voltage = 208
    evse_type = 'AeroVironment'

    # Define the sets of EVSEs in the Caltech ACN.
    AB_ids = ['AB-{0}'.format(i) for i in range(evse_per_phase)]
    BC_ids = ['BC-{0}'.format(i) for i in range(evse_per_phase)]
    CA_ids = ['CA-{0}'.format(i) for i in range(evse_per_phase)]

    # Add Caltech EVSEs
    for evse_id in AB_ids:
        network.register_evse(acnsim.get_evse_by_type(evse_id, evse_type), voltage, 30)
    for evse_id in BC_ids:
        network.register_evse(acnsim.get_evse_by_type(evse_id, evse_type), voltage, -90)
    for evse_id in CA_ids:
        network.register_evse(acnsim.get_evse_by_type(evse_id, evse_type), voltage, 150)

    # Add Caltech Constraint Set
    AB = acnsim.Current(AB_ids)
    BC = acnsim.Current(BC_ids)
    CA = acnsim.Current(CA_ids)

    # Define intermediate currents
    I3a = AB - CA
    I3b = BC - AB
    I3c = CA - BC
    I2a = (1 / 4) * (I3a - I3c)
    I2b = (1 / 4) * (I3b - I3a)
    I2c = (1 / 4) * (I3c - I3b)

    # Build constraint set
    primary_side_constr = transformer_cap * 1000 / 3 / 277
    secondary_side_constr = transformer_cap * 1000 / 3 / 120
    network.add_constraint(I3a, secondary_side_constr, name='Secondary A')
    network.add_constraint(I3b, secondary_side_constr, name='Secondary B')
    network.add_constraint(I3c, secondary_side_constr, name='Secondary C')
    network.add_constraint(I2a, primary_side_constr, name='Primary A')
    network.add_constraint(I2b, primary_side_constr, name='Primary B')
    network.add_constraint(I2c, primary_side_constr, name='Primary C')

    return network


## Experiments

In these experiments we will run a simulation for each system configuration can compare the results on key metrics.

In [4]:
# How long each time discrete time interval in the simulation should be.
PERIOD = 5  # minutes

# Voltage of the network.
VOLTAGE = 208  # volts

# Default maximum charging rate for each EV battery.
DEFAULT_BATTERY_POWER = 6.6 # kW

**Network Options**

We consider five infrastrucuture configurations.

In [5]:
# Network of 102 Level-1 EVSEs with a 200 kW Transformer
level_1 = level_1_network(transformer_cap=200, evse_per_phase=34)

# Network of 30 Level-2 EVSEs with a 200 kW Transformer
level_2_200kW_30 = level_2_network(transformer_cap=200, evse_per_phase=10)

# Network of 102 Level-2 EVSEs with a 200 kW Transformer
level_2_200kW_102 = level_2_network(transformer_cap=200, evse_per_phase=34)

# Network of 102 Level-2 EVSEs with a 670 kW Transformer
level_2_670kW_102 = level_2_network(transformer_cap=670, evse_per_phase=34)

# Network of 201 Level-2 EVSEs with a 200 kW Transformer
level_2_200kW_201 = level_2_network(transformer_cap=200, evse_per_phase=67)

**Events**

We assume that our site will have a usage profile similar to JPL, so we use a Gaussian Mixture Model trained on data from weekdays at JPL to generate events for this experiment. We assume that the site will be closed on weekends, so no charging will occur. 

In [6]:
def get_synth_events(sessions_per_day):
    gmm = pickle.load(open('./data/jpl_weekday_40.pkl', 'rb'))

    # Generate a list of the number of sessions to draw for each day.
    # This generates 30 days of charging demands.
    num_evs = [0]*2 + [sessions_per_day]*5 + [0]*2 + [sessions_per_day]*5 + [0]*2 + \
              [sessions_per_day]*5 + [0]*2 + [sessions_per_day]*5 + [0]*2

    # Note that because we are drawing from a distribution, some sessions will be
    # invalid, we ignore these sessions and remove the corresponding plugin events. 
    gen = GaussianMixtureEvents(pretrained_model=gmm)

    synth_events = gen.generate_events(num_evs, PERIOD, VOLTAGE, DEFAULT_BATTERY_POWER)
    return synth_events

In [7]:
# Events with 100 EVs per weekday
sessions_100 = get_synth_events(100)

# Events with 200 EVs per weekday
sessions_200 = get_synth_events(200)

/opt/conda/lib/python3.8/site-packages/sklearn/utils/deprecation.py:143: FutureWarning: The sklearn.mixture.gaussian_mixture module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.mixture. Anything that cannot be imported from sklearn.mixture is now part of the private API.
  warnings.warn(message, FutureWarning)
/opt/conda/lib/python3.8/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator GaussianMixture from version 0.20.2 when using version 0.23.2. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(


In [8]:
def run_experiment(network, algorithm, events):
    """ Run simulation for the events defined previously and the specified
        network / algorithm / events. 
    """
    # Timezone of the ACN we are using.
    timezone = pytz.timezone('America/Los_Angeles')
    
    # Start and End times are used when collecting data.
    start = timezone.localize(datetime(2019, 6, 1))
    end = timezone.localize(datetime(2019, 7, 1))
    
    sch = deepcopy(algorithm)
    cn = deepcopy(network)
    signals = {'tariff': TimeOfUseTariff('sce_tou_ev_4_march_2019')}

    sim = acnsim.Simulator(cn, sch, events, start, period=PERIOD, verbose=False, signals=signals)
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
    sim.run()

    r = {'proportion_of_energy_delivered': acnsim.proportion_of_energy_delivered(sim),
         'energy_delivered': sum(ev.energy_delivered for ev in sim.ev_history.values()),
         'num_swaps': cn.swaps,
         'num_never_charged': cn.never_charged,
         'energy_cost': acnsim.energy_cost(sim),
         'demand_charge': acnsim.demand_charge(sim)
         }
    r['total_cost'] = r['energy_cost'] + r['demand_charge']
    r['$/kWh'] = r['total_cost'] / r['energy_delivered']
    return r

**Define Algorithms**

We consider Uncontrolled, Least-Laxity First and Model Predictive Control (cost-minimization).  

In [9]:
uncontrolled = algorithms.UncontrolledCharging()
llf = algorithms.SortedSchedulingAlgo(algorithms.least_laxity_first)

In [10]:
def days_remaining_scale_demand_charge(rates, infrastructure, interface,
                                       baseline_peak=0, **kwargs):
    day_index = interface.current_time // ((60 / interface.period) * 24)
    days_in_month = 30
    day_index = min(day_index, days_in_month - 1)
    scale = 1 / (days_in_month - day_index)
    dc = adacharge.demand_charge(rates, infrastructure, interface, baseline_peak, **kwargs)
    return scale * dc

In [11]:
cost_min_obj = [adacharge.ObjectiveComponent(adacharge.total_energy, 1000),
                adacharge.ObjectiveComponent(adacharge.tou_energy_cost),
                adacharge.ObjectiveComponent(days_remaining_scale_demand_charge),
                adacharge.ObjectiveComponent(adacharge.quick_charge, 1e-4),
                adacharge.ObjectiveComponent(adacharge.equal_share, 1e-12)
               ]
cost_min = adacharge.AdaptiveSchedulingAlgorithm(cost_min_obj, solver="ECOS", quantize=True, reallocate=True, peak_limit=1000, max_recompute=1)

**Run Experiments (100 EVs)**

In [12]:
level1_unctrl_100 = run_experiment(level_1, uncontrolled, deepcopy(sessions_100))

In [13]:
level2_200kW_untrl_100 = run_experiment(level_2_200kW_30, uncontrolled, deepcopy(sessions_100))

/opt/conda/lib/python3.8/site-packages/acnportal/contrib/acnsim/network/stochastic_network.py:81: UserWarning: Tried to remove EV with session_id session_55 which was not present at station BC-6. Found no EV instead.
  warnings.warn(
/opt/conda/lib/python3.8/site-packages/acnportal/contrib/acnsim/network/stochastic_network.py:81: UserWarning: Tried to remove EV with session_id session_21 which was not present at station BC-5. Found no EV instead.
  warnings.warn(
/opt/conda/lib/python3.8/site-packages/acnportal/contrib/acnsim/network/stochastic_network.py:81: UserWarning: Tried to remove EV with session_id session_54 which was not present at station BC-0. Found no EV instead.
  warnings.warn(
/opt/conda/lib/python3.8/site-packages/acnportal/contrib/acnsim/network/stochastic_network.py:81: UserWarning: Tried to remove EV with session_id session_80 which was not present at station AB-0. Found no EV instead.
  warnings.warn(
/opt/conda/lib/python3.8/site-packages/acnportal/contrib/acnsim/

In [14]:
level2_670kW_unctrl_100 = run_experiment(level_2_670kW_102, uncontrolled, deepcopy(sessions_100))

In [15]:
level2_200kW_llf_100 = run_experiment(level_2_200kW_102, llf, deepcopy(sessions_100))

In [ ]:
level2_200kW_cost_min_100 = run_experiment(level_2_200kW_102, cost_min, deepcopy(sessions_100))

/opt/conda/lib/python3.8/site-packages/cvxpy/problems/problem.py:1060: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  warnings.warn(


**Run Experiments (200 EVs)**

In [ ]:
level1_unctrl_200 = run_experiment(level_1, uncontrolled, deepcopy(sessions_200))

In [ ]:
level2_200kW_untrl_200 = run_experiment(level_2_200kW_30, uncontrolled, deepcopy(sessions_200))

In [ ]:
level2_670kW_unctrl_200 = run_experiment(level_2_670kW_102, uncontrolled, deepcopy(sessions_200))

In [ ]:
level2_200kW_llf_200 = run_experiment(level_2_200kW_102, llf, deepcopy(sessions_200))

In [ ]:
level2_200kW_cost_min_200 = run_experiment(level_2_200kW_102, cost_min, deepcopy(sessions_200))

In [ ]:
level2_200kW_cost_min_201_200 = run_experiment(level_2_200kW_201, cost_min, deepcopy(sessions_200))

### Analyze Results

In [28]:
import pandas as pd
evs_100 = pd.DataFrame({
    'Level 1: Unctrl: 200 kW : 102 EVSEs': level1_unctrl_100,
    'Level 2: Unctrl: 200 kW : 30 EVSEs':  level2_200kW_untrl_100,
    'Level 2: Unctrl: 670 kW : 102 EVSEs': level2_670kW_unctrl_100,
    'Level 2: LLF: 200 kW : 102 EVSEs': level2_200kW_llf_100,
    'Level 2: Min Cost: 200 kW : 102 EVSEs': level2_200kW_cost_min_100
})
evs_100

,Level 1: Unctrl: 200 kW : 102 EVSEs,Level 2: Unctrl: 200 kW : 30 EVSEs,Level 2: Unctrl: 670 kW : 102 EVSEs,Level 2: LLF: 200 kW : 102 EVSEs,Level 2: Min Cost: 200 kW : 102 EVSEs
proportion_of_energy_delivered,0.754183,0.996589,0.998345,0.996922,0.995691
energy_delivered,17561.847152,23206.505937,23247.392937,23214.261333,23185.604630
num_swaps,0.000000,1104.000000,0.000000,0.000000,0.000000
num_never_charged,0.000000,4.000000,0.000000,0.000000,0.000000
energy_cost,2621.562284,2822.094059,2728.630970,2796.926926,2856.453056
demand_charge,2176.680895,3070.980000,4895.898192,3076.811760,2469.192000
total_cost,4798.243179,5893.074059,7624.529162,5873.738686,5325.645056
$/kWh,0.273220,0.253941,0.327974,0.253023,0.229696


In [29]:
evs_200 = pd.DataFrame({
    'Level 1: Unctrl: 200 kW : 102 EVSEs': level1_unctrl_200,
    'Level 2: Unctrl: 200 kW : 30 EVSEs':  level2_200kW_untrl_200,
    'Level 2: Unctrl: 670 kW : 102 EVSEs': level2_670kW_unctrl_200,
    'Level 2: LLF: 200 kW : 102 EVSEs': level2_200kW_llf_200,
    'Level 2: Min Cost: 200 kW : 102 EVSEs': level2_200kW_cost_min_200,
    'Level 2: Min Cost: 200 kW : 201 EVSEs': level2_200kW_cost_min_201_200
})
evs_200

,Level 1: Unctrl: 200 kW : 102 EVSEs,Level 2: Unctrl: 200 kW : 30 EVSEs,Level 2: Unctrl: 670 kW : 102 EVSEs,Level 2: LLF: 200 kW : 102 EVSEs,Level 2: Min Cost: 200 kW : 102 EVSEs,Level 2: Min Cost: 200 kW : 201 EVSEs
proportion_of_energy_delivered,0.736772,0.915827,0.998560,0.880725,0.871940,0.987986
energy_delivered,34076.724373,42358.244240,46184.774552,40734.744667,40328.397983,45695.696905
num_swaps,1195.000000,2971.000000,1095.000000,1443.000000,1425.000000,0.000000
num_never_charged,19.000000,183.000000,0.000000,309.000000,332.000000,0.000000
energy_cost,5298.150273,6797.589344,5464.878676,6626.676764,6019.390036,7239.775793
demand_charge,3037.478400,3070.980000,9661.213390,3077.680320,3098.525760,3100.262880
total_cost,8335.628673,9868.569344,15126.092066,9704.357084,9117.915796,10340.038673
$/kWh,0.244614,0.232979,0.327513,0.238233,0.226092,0.226280


In [31]:
evs_100.to_csv("results/100_EV_comparison.csv")

In [42]:
pd.DataFrame({
    'Level 1: Unctrl: 200 kW : 102 EVSEs': level1_unctrl_200,
    'Level 2: Unctrl: 200 kW : 30 EVSEs':  level2_200kW_untrl_200,
    'Level 2: Unctrl: 670 kW : 102 EVSEs': level2_670kW_unctrl_200,
    'Level 2: LLF: 200 kW : 102 EVSEs': level2_200kW_llf_200,
    'Level 2: Min Cost: 200 kW : 102 EVSEs': level2_200kW_cost_min_200,
    'Level 2: Min Cost: 200 kW : 201 EVSEs': level2_200kW_cost_min_201_200
})

,Level 1: Unctrl: 200 kW : 102 EVSEs,Level 2: Unctrl: 200 kW : 30 EVSEs,Level 2: Unctrl: 670 kW : 102 EVSEs,Level 2: LLF: 200 kW : 102 EVSEs,Level 2: Min Cost: 200 kW : 102 EVSEs,Level 2: Min Cost: 200 kW : 201 EVSEs
proportion_of_energy_delivered,0.738110,0.923110,0.998562,0.877000,0.866881,0.993480
energy_delivered,33795.857624,42266.450965,45721.146312,40155.198000,39691.865510,45488.456769
num_swaps,1175.000000,2982.000000,1097.000000,1417.000000,1405.000000,0.000000
num_never_charged,15.000000,171.000000,0.000000,333.000000,347.000000,0.000000
energy_cost,5250.960930,6807.370107,5476.685593,6496.089770,5890.720942,7209.362828
demand_charge,3037.478400,3070.980000,9292.122362,3076.811760,3100.262880,3100.262880
total_cost,8288.439330,9878.350107,14768.807955,9572.901530,8990.983822,10309.625708
$/kWh,0.245250,0.233716,0.323019,0.238398,0.226520,0.226643


In [32]:
evs_200.to_csv("results/200_EV_comparison.csv")

In [13]:
import pandas as pd
pd.DataFrame({'Level 1: Unctrl: 200 kW : 102 EVSEs': level1_uncontrolled,
              'Level 2: Unctrl: 200 kW : 30 EVSEs':  level2_200kW_uncontrolled,
              'Level 2: Unctrl: 670 kW : 102 EVSEs': level2_670kW_uncontrolled,
              'Level 2: LLF: 200 kW : 102 EVSEs': level2_200kW_llf})

,Level 1: Unctrl: 200 kW : 102 EVSEs,Level 2: Unctrl: 200 kW : 30 EVSEs,Level 2: Unctrl: 670 kW : 102 EVSEs,Level 2: LLF: 200 kW : 102 EVSEs
proportion_of_energy_delivered,0.759317,0.632315,0.999473,0.997959
energy_delivered,17430.906777,14515.451255,22943.933367,22909.180000
num_swaps,0.000000,898.000000,0.000000,0.000000
num_never_charged,0.000000,456.000000,0.000000,0.000000
energy_cost,2625.680014,2186.644529,2757.569682,2824.104461
demand_charge,2173.881600,2763.882000,5033.572243,3076.811760
total_cost,4799.561614,4950.526529,7791.141925,5900.916221
$/kWh,0.275348,0.341052,0.339573,0.257579


From the above table we can see that smart charging using even a simple LLF algorithm has significant benefits over Uncontrolled Level-1 charging in terms of amount of demand met. It also requires far less infrastructure than Uncontrolled Level-2 charging with the same number of EVSEs, and without requiring users to swap spaces mid-day. 